In [1]:
from utils import base_configs, deps, tr_va_te_split
from utils import tr_va_te_split
from utils.helpers import dir_helpers, rw_csv_helpers, feature_distr_helpers, feature_transform_helpers

/home/ppanta/puru_proj/proj_v0/hints6_v0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys, math
import import_ipynb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### 0 Data load

In [3]:
df_hints6 = "ip/3_cleanedEncoded/hints6_public_filtered_v1_cleaned_encoded.csv"
df_hints7 = "ip/3_cleanedEncoded/....csv"
df_hints6_7 = "ip/3_cleanedEncoded/hints6_7_cleaned_encoded.csv"
df_orig = rw_csv_helpers.read_csv_file(df_hints6_7, verbose = 1)

Loaded: /home/ppanta/puru_proj/proj_v0/hints6_v0/ip/3_cleanedEncoded/hints6_7_cleaned_encoded.csv
────────────────────────────────────────────────────────────────────────────────
Shape: (10581, 26)
────────────────────────────────────────────────────────────────────────────────
All columns: ['FreqGoProvider', 'Deaf', 'MedConditions_Diabetes', 'MedConditions_HighBP', 'MedConditions_HeartCondition', 'MedConditions_LungDisease', 'MedConditions_Depression', 'AverageTimeSitting', 'EverHadCancer', 'Age', 'BirthGender', 'BMI', 'PHQ4', 'WeeklyMinutesModerateExercise', 'AvgDrinksPerWeek', 'GeneralHealth_Excellent', 'GeneralHealth_VeryGood', 'GeneralHealth_Good', 'GeneralHealth_Fair', 'GeneralHealth_Poor', 'smokeStat_Current', 'smokeStat_Former', 'smokeStat_Never', 'eCigUse_Current', 'eCigUse_Former', 'eCigUse_Never']


In [4]:
counts = feature_distr_helpers.count01(df_orig.copy(), "MedConditions_HeartCondition", verbose=1)

Counts for column 'MedConditions_HeartCondition' (only 0 and 1):
MedConditions_HeartCondition
0    9573
1    1008
Name: count, dtype: int64
Total (0/1 only): 10581


### 1 Train - Validation - Test split

In [6]:
df_orig = df_orig.copy()
target_col = "MedConditions_HeartCondition"
CONFIGS = base_configs.get_base_configs()

X = df_orig.drop(columns=[target_col])
y = df_orig[target_col]

result = tr_va_te_split.data_preprocessing(
    CONFIGS=CONFIGS,
    verbose=0,       # 0, 1, or 2
    X=X, y=y,
    balance_method="adasyn",  # or 'smote', 'smoteenn', 'none'
    balance_kwargs={"n_neighbors": 5}  # optional
)

### 2 Train - Validation - Test value assignments

In [7]:
print_result = feature_distr_helpers.print_shapes_and_features(result, verbose = 1)

# Value assignments for further calculations
X_train_res_scaled = result['X_train_res_scaled']
X_val_scaled       = result['X_val_scaled']
X_test_scaled      = result['X_test_scaled']

X_train     = result['X_train']
X_train_res = result['X_train_res']
X_val       = result['X_val']
X_test      = result['X_test']

y_train     = result['y_train']
y_train_res = result['y_train_res']
y_val       = result['y_val']
y_test      = result['y_test']

features    = result['features']


X_train shape:            X = (6348, 25),      y = (6348,)
X_train_res shape:        X = (11494, 25),  y = (11494,)
X_val shape:              X = (2116, 25),        y = (2116,)
X_test shape:             X = (2117, 25),       y = (2117,)
X_train_res_scaled shape: X = (11494, 25)
X_val_scaled shape:       X = (2116, 25)
X_test_scaled shape:      X = (2117, 25)
features length:          n = 25


### 3 Random forest Train-Validate-Test

In [9]:
from utils import pipeline_rf as pipeline_rf

In [10]:
model_rf, results_rf = pipeline_rf.run_rf_pipeline(
    X_train_res_scaled, y_train_res,
    X_val_scaled, y_val,
    X_test_scaled, y_test,
    features,
    class_weight="balanced",
    threshold_strategy= "f1" #"balanced_accuracy", #"f1", #"cost",
)


AttributeError: module 'utils.pipeline_rf' has no attribute 'run_rf_pipeline'

In [11]:
# in your notebook
from utils.ml_lr import run_lr_pipeline_compat, export_lr_artifacts

model_lr, results_lr = run_lr_pipeline_compat(
    X_train_res_scaled, y_train_res,
    X_val_scaled, y_val,
    X_test_scaled, y_test,
    features,
    class_weight="balanced",
    threshold_strategy="f1",
)
print(results_lr["test"]["metrics"])

export_lr_artifacts(results_lr, outdir="op/2_ml_predict/lr_run_001")


{'accuracy': 0.7800616649537513, 'balanced_accuracy': np.float64(0.7441109852774632), 'f1': 0.37058823529411766, 'precision': 0.252, 'recall': 0.7, 'roc_auc': np.float64(0.8001635837422927), 'avg_precision': np.float64(0.31520548198724685)}

=== Logistic Regression Metrics ===
      accuracy  balanced_accuracy      f1  precision  recall  roc_auc  avg_precision
val     0.7729             0.7170  0.3481     0.2379  0.6484   0.7691         0.2594
test    0.7801             0.7441  0.3706     0.2520  0.7000   0.8002         0.3152

Chosen threshold (from VAL): 0.2000


{'val_sweep': 'op/2_ml_predict/lr_run_001/lr_val_sweep_20251015_144449.csv',
 'test_sweep': 'op/2_ml_predict/lr_run_001/lr_test_sweep_20251015_144449.csv',
 'val_metrics': 'op/2_ml_predict/lr_run_001/lr_val_metrics_20251015_144449.csv',
 'test_metrics': 'op/2_ml_predict/lr_run_001/lr_test_metrics_20251015_144449.csv',
 'combined_metrics': 'op/2_ml_predict/lr_run_001/lr_combined_metrics_20251015_144449.csv',
 'val_report': 'op/2_ml_predict/lr_run_001/lr_val_report_20251015_144449.txt',
 'test_report': 'op/2_ml_predict/lr_run_001/lr_test_report_20251015_144449.txt'}

In [12]:
from datetime import datetime
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
outdir = f"op/2_ml_predict/lr_run_{ts}"

paths = export_lr_artifacts(
    results_lr,
    outdir=outdir,
    prefix="lr",
    timestamp=ts,
    include_preds=True,
    print_metrics=True   # <-- prints val & test metrics to console
)



=== Logistic Regression Metrics ===
      accuracy  balanced_accuracy      f1  precision  recall  roc_auc  avg_precision
val     0.7729             0.7170  0.3481     0.2379  0.6484   0.7691         0.2594
test    0.7801             0.7441  0.3706     0.2520  0.7000   0.8002         0.3152

Chosen threshold (from VAL): 0.2000


In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_score, recall_score,
    f1_score, balanced_accuracy_score, matthews_corrcoef, brier_score_loss,
    log_loss, confusion_matrix
)

# ============================================================
# Pretty table helpers
# ============================================================
def _is_notebook():
    try:
        from IPython import get_ipython
        return "IPKernelApp" in get_ipython().config
    except Exception:
        return False

def style_summary(df: pd.DataFrame, caption: str = ""):
    """Return a styled DataFrame with nice formatting for notebooks."""
    # Columns where "higher is better" (we'll color the max)
    higher_better = [
        "ROC_AUC", "PR_AUC", "Accuracy", "Precision", "Recall", "F1",
        "Balanced_Accuracy", "MCC"
    ]
    # Columns where "lower is better"
    lower_better = [
        "Brier", "LogLoss", "ExpectedCost", "CostPerCase", "FP", "FN"
    ]

    # Round numeric cols
    fmt = {col: "{:.4f}" for col in df.columns if df[col].dtype.kind in "fc"}
    fmt.update({col: "{:.0f}" for col in ["TN","FP","FN","TP"] if col in df.columns})

    s = (
        df.style
        .set_caption(caption)
        .format(fmt)
        .set_properties(**{"text-align": "center"})
        .set_table_styles([
            {"selector":"caption", "props":[("caption-side","top"),("font-weight","bold"),("font-size","110%")]},
            {"selector":"th", "props":[("text-align","center"),("font-weight","bold")]},
            {"selector":"td", "props":[("padding","6px 10px")]}
        ])
    )

    # Highlight best values
    for col in higher_better:
        if col in df.columns:
            s = s.highlight_max(subset=[col], color="#d2f5d2")
    for col in lower_better:
        if col in df.columns:
            s = s.highlight_min(subset=[col], color="#d2f5d2")

    return s

def print_or_display(df_val: pd.DataFrame, df_tst: pd.DataFrame):
    """Display nicely in notebooks; otherwise print with tabulate or fallback."""
    if _is_notebook():
        from IPython.display import display
        display(style_summary(df_val, "VALIDATION (threshold chosen here)"))
        display(style_summary(df_tst, "TEST (evaluated at the same threshold)"))
    else:
        # console-friendly pretty print
        try:
            from tabulate import tabulate
            print("\nVALIDATION (threshold chosen here)")
            print(tabulate(df_val, headers="keys", tablefmt="github", showindex=False))
            print("\nTEST (evaluated at the same threshold)")
            print(tabulate(df_tst, headers="keys", tablefmt="github", showindex=False))
        except Exception:
            # plain fallback
            print("\nVALIDATION (threshold chosen here)")
            print(df_val.to_string(index=False))
            print("\nTEST (evaluated at the same threshold)")
            print(df_tst.to_string(index=False))

# ============================================================
# Core helpers
# ============================================================
def proj_simplex(v):
    """Project vector v onto the probability simplex {w >= 0, sum(w)=1}."""
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, len(u)+1) > (cssv - 1))[0][-1]
    theta = (cssv[rho] - 1) / (rho + 1.0)
    return np.maximum(v - theta, 0.0)

def solve_simplex_brier(P, y, iters=1500, lr=0.5, seed=42):
    """
    Learn convex weights that minimize Brier score on validation.
    P: (n_val, k) matrix of validation probabilities (one column per model)
    y: (n_val,) array of true labels {0,1}
    """
    rng = np.random.default_rng(seed)
    k = P.shape[1]
    w = np.ones(k) / k  # start uniform
    for _ in range(iters):
        grad = (2.0 / len(y)) * P.T.dot(P.dot(w) - y)
        w = proj_simplex(w - lr * grad)
    return w

def sweep_thresholds(y_true, y_prob, grid=None):
    if grid is None: grid = np.linspace(0, 1, 101)
    rows = []
    for t in grid:
        y_pred = (y_prob >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        n = len(y_true)
        rows.append({
            "thr": t,
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "f1": f1_score(y_true, y_pred, zero_division=0),
            "acc": (tn+tp)/n,
            "bacc": balanced_accuracy_score(y_true, y_pred),
            "fp": fp, "fn": fn
        })
    return pd.DataFrame(rows)

def find_best_threshold(y_true, y_prob,
                        strategy="cost",  # "cost" | "f1" | "balanced_accuracy" | "accuracy"
                        fp_cost=1.0, fn_cost=4.0,
                        grid=None):
    sw = sweep_thresholds(y_true, y_prob, grid)
    if strategy == "f1":
        idx = sw["f1"].idxmax()
    elif strategy == "balanced_accuracy":
        idx = sw["bacc"].idxmax()
    elif strategy == "accuracy":
        idx = sw["acc"].idxmax()
    elif strategy == "cost":
        cost = sw["fp"]*fp_cost + sw["fn"]*fn_cost
        sw["cost"] = cost
        idx = cost.idxmin()
    else:
        raise ValueError("Unknown strategy")
    return float(sw.loc[idx, "thr"]), sw

def evaluate_at_threshold(y_true, y_prob, thr):
    """Return both threshold-free and at-threshold metrics."""
    y_pred = (y_prob >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    n = len(y_true)
    return {
        "Threshold": thr,
        "ROC_AUC": roc_auc_score(y_true, y_prob),
        "PR_AUC": average_precision_score(y_true, y_prob),
        "Accuracy": (tn + tp) / n,
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "Balanced_Accuracy": balanced_accuracy_score(y_true, y_pred),
        "MCC": matthews_corrcoef(y_true, y_pred),
        "Brier": brier_score_loss(y_true, y_prob),
        "LogLoss": log_loss(y_true, y_prob),
        "TN": tn, "FP": fp, "FN": fn, "TP": tp
    }

# ============================================================
# Base models (LogReg + RandomForest)
# ============================================================
def fit_logreg(X_train, y_train, random_state=42):
    lr = LogisticRegression(
        max_iter=1000, solver="liblinear",
        class_weight="balanced", random_state=random_state
    )
    lr.fit(X_train, y_train)
    return lr

def fit_random_forest(X_train, y_train, random_state=42):
    rf = RandomForestClassifier(
        n_estimators=600, max_depth=None, max_features="sqrt",
        min_samples_leaf=1, class_weight=None,   # set to "balanced" if desired
        n_jobs=-1, random_state=random_state
    )
    rf.fit(X_train, y_train)
    return rf

# ============================================================
# Train, weight, threshold, evaluate, pretty-print
# (uses your X_* / y_* variables)
# ============================================================
# 1) Train
lr = fit_logreg(X_train_res_scaled, y_train_res)
rf = fit_random_forest(X_train_res_scaled, y_train_res)

# 2) Probabilities
val_lr   = lr.predict_proba(X_val_scaled)[:, 1]
val_rf   = rf.predict_proba(X_val_scaled)[:, 1]
test_lr  = lr.predict_proba(X_test_scaled)[:, 1]
test_rf  = rf.predict_proba(X_test_scaled)[:, 1]

# 3) Learn convex weights on validation
P_val  = np.column_stack([val_lr, val_rf])   # (n_val, 2)
P_test = np.column_stack([test_lr, test_rf]) # (n_test, 2)
w_star = solve_simplex_brier(P_val, y_val, iters=1500, lr=0.5)
print("Ensemble weights (LR, RF):", np.round(w_star, 4), "sum=", round(w_star.sum(), 4))

val_ens  = P_val.dot(w_star)
test_ens = P_test.dot(w_star)

# 4) Choose threshold on VALIDATION ENSEMBLE
THRESH_STRATEGY = "cost"    # "cost" | "f1" | "balanced_accuracy" | "accuracy"
FP_COST, FN_COST = 1.0, 4.0
thr, sweep = find_best_threshold(y_val, val_ens,
                                 strategy=THRESH_STRATEGY,
                                 fp_cost=FP_COST, fn_cost=FN_COST)
print("Chosen threshold (validation):", round(thr, 3))

# 5) Evaluate VAL + TEST at same threshold
res_lr_val   = evaluate_at_threshold(y_val,  val_lr,   thr)
res_rf_val   = evaluate_at_threshold(y_val,  val_rf,   thr)
res_ens_val  = evaluate_at_threshold(y_val,  val_ens,  thr)
res_lr_test  = evaluate_at_threshold(y_test, test_lr,  thr)
res_rf_test  = evaluate_at_threshold(y_test, test_rf,  thr)
res_ens_test = evaluate_at_threshold(y_test, test_ens, thr)

summary_val = pd.DataFrame([
    {"Model":"LogisticReg",      **res_lr_val},
    {"Model":"RandomForest",     **res_rf_val},
    {"Model":"Ensemble(LR+RF)",  **res_ens_val},
])

summary_test = pd.DataFrame([
    {"Model":"LogisticReg",      **res_lr_test},
    {"Model":"RandomForest",     **res_rf_test},
    {"Model":"Ensemble(LR+RF)",  **res_ens_test},
])

# Add cost columns
summary_val["ExpectedCost"] = summary_val["FP"]*FP_COST + summary_val["FN"]*FN_COST
summary_test["ExpectedCost"] = summary_test["FP"]*FP_COST + summary_test["FN"]*FN_COST
summary_val["CostPerCase"]  = summary_val["ExpectedCost"] / len(y_val)
summary_test["CostPerCase"] = summary_test["ExpectedCost"] / len(y_test)

# Select + order columns for display
cols_show = [
    "Model","Threshold","ROC_AUC","PR_AUC","Accuracy","Precision","Recall","F1",
    "Balanced_Accuracy","MCC","Brier","LogLoss","TN","FP","FN","TP","ExpectedCost","CostPerCase"
]
summary_val = summary_val[cols_show].round(4)
summary_test = summary_test[cols_show].round(4)

# 6) Pretty print (notebook Styler or console tabulate)
print_or_display(summary_val, summary_test)

# 7) (Optional) Export to CSV
# summary_val.to_csv("validation_summary.csv", index=False)
# summary_test.to_csv("test_summary.csv", index=False)
# pd.DataFrame(sweep).to_csv("threshold_sweep_validation.csv", index=False)


Ensemble weights (LR, RF): [0.9287 0.0713] sum= 1.0
Chosen threshold (validation): 0.29


AttributeError: The '.style' accessor requires jinja2

In [13]:
# Sweep mix to see why LR dominates
alphas = np.linspace(0, 1, 21)
briers = [brier_score_loss(y_val, a*val_lr + (1-a)*val_rf) for a in alphas]
best_a = alphas[np.argmin(briers)]
print("Best alpha for LR share on val:", round(best_a,3))


Best alpha for LR share on val: 0.95


In [14]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, HistGradientBoostingClassifier,
    GradientBoostingClassifier, ExtraTreesClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_score, recall_score,
    f1_score, balanced_accuracy_score, matthews_corrcoef, brier_score_loss,
    log_loss, confusion_matrix
)

# ============================================================
# Pretty table helpers
# ============================================================
def _is_notebook():
    try:
        from IPython import get_ipython
        return "IPKernelApp" in get_ipython().config
    except Exception:
        return False

def style_summary(df: pd.DataFrame, caption: str = ""):
    higher_better = [
        "ROC_AUC","PR_AUC","Accuracy","Precision","Recall","F1",
        "Balanced_Accuracy","MCC"
    ]
    lower_better = ["Brier","LogLoss","ExpectedCost","CostPerCase","FP","FN"]
    fmt = {col: "{:.4f}" for col in df.columns if df[col].dtype.kind in "fc"}
    for c in ["TN","FP","FN","TP"]:
        if c in df.columns: fmt[c] = "{:.0f}"
    s = (df.style
         .set_caption(caption)
         .format(fmt)
         .set_properties(**{"text-align":"center"})
         .set_table_styles([
             {"selector":"caption","props":[("caption-side","top"),("font-weight","bold"),("font-size","110%")]},
             {"selector":"th","props":[("text-align","center"),("font-weight","bold")]},
             {"selector":"td","props":[("padding","6px 10px")]}
         ]))
    for col in higher_better:
        if col in df.columns: s = s.highlight_max(subset=[col], color="#d2f5d2")
    for col in lower_better:
        if col in df.columns: s = s.highlight_min(subset=[col], color="#d2f5d2")
    return s

def print_or_display(df_val: pd.DataFrame, df_tst: pd.DataFrame):
    """Notebook-safe display without using .style or jinja2."""
    if _is_notebook():
        from IPython.display import display, HTML
        display(HTML("<h4>VALIDATION (threshold chosen here)</h4>"))
        display(df_val)
        display(HTML("<div style='margin:10px 0;border-top:2px solid #888;'></div>"))
        display(HTML("<h4>TEST (evaluated at the same threshold)</h4>"))
        display(df_tst)
    else:
        try:
            from tabulate import tabulate
            print("\nVALIDATION (threshold chosen here)")
            print(tabulate(df_val, headers="keys", tablefmt="github", showindex=False))
            print("\nTEST (evaluated at the same threshold)")
            print(tabulate(df_tst, headers="keys", tablefmt="github", showindex=False))
        except Exception:
            print("\nVALIDATION (threshold chosen here)")
            print(df_val.to_string(index=False))
            print("\nTEST (evaluated at the same threshold)")
            print(df_tst.to_string(index=False))


# ============================================================
# Core helpers (simplex weighting + metrics)
# ============================================================
def proj_simplex(v):
    """Project vector v onto {w >= 0, sum(w)=1}."""
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, len(u)+1) > (cssv - 1))[0][-1]
    theta = (cssv[rho] - 1) / (rho + 1.0)
    return np.maximum(v - theta, 0.0)

def solve_simplex_brier(P, y, iters=1500, lr=0.5, seed=42):
    """
    Learn convex weights that minimize Brier score on validation.
    P: (n_val, k) matrix of validation probabilities (columns = models)
    y: (n_val,) labels {0,1}
    """
    rng = np.random.default_rng(seed)
    k = P.shape[1]
    w = np.ones(k) / k
    for _ in range(iters):
        grad = (2.0 / len(y)) * P.T.dot(P.dot(w) - y)
        w = proj_simplex(w - lr * grad)
    return w

def sweep_thresholds(y_true, y_prob, grid=None):
    if grid is None: grid = np.linspace(0, 1, 101)
    rows = []
    for t in grid:
        y_pred = (y_prob >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        n = len(y_true)
        rows.append({
            "thr": t,
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "f1": f1_score(y_true, y_pred, zero_division=0),
            "acc": (tn+tp)/n,
            "bacc": balanced_accuracy_score(y_true, y_pred),
            "fp": fp, "fn": fn
        })
    return pd.DataFrame(rows)

def find_best_threshold(y_true, y_prob,
                        strategy="cost",  # "cost" | "f1" | "balanced_accuracy" | "accuracy"
                        fp_cost=1.0, fn_cost=4.0,
                        grid=None):
    sw = sweep_thresholds(y_true, y_prob, grid)
    if strategy == "f1":
        idx = sw["f1"].idxmax()
    elif strategy == "balanced_accuracy":
        idx = sw["bacc"].idxmax()
    elif strategy == "accuracy":
        idx = sw["acc"].idxmax()
    elif strategy == "cost":
        sw["cost"] = sw["fp"]*fp_cost + sw["fn"]*fn_cost
        idx = sw["cost"].idxmin()
    else:
        raise ValueError("Unknown strategy")
    return float(sw.loc[idx, "thr"]), sw

def evaluate_at_threshold(y_true, y_prob, thr):
    y_pred = (y_prob >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    n = len(y_true)
    return {
        "Threshold": thr,
        "ROC_AUC": roc_auc_score(y_true, y_prob),
        "PR_AUC": average_precision_score(y_true, y_prob),
        "Accuracy": (tn + tp) / n,
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "Balanced_Accuracy": balanced_accuracy_score(y_true, y_pred),
        "MCC": matthews_corrcoef(y_true, y_pred),
        "Brier": brier_score_loss(y_true, y_prob),
        "LogLoss": log_loss(y_true, y_prob),
        "TN": tn, "FP": fp, "FN": fn, "TP": tp
    }

# ============================================================
# Model zoo — adjust as you like
# (All trained on scaled features you already have.)
# ============================================================
def get_model_zoo(random_state=42):
    models = {
        "LogisticReg": LogisticRegression(
            max_iter=1000, solver="liblinear", class_weight="balanced",
            random_state=random_state
        ),
        "RandomForest": RandomForestClassifier(
            n_estimators=600, max_depth=None, max_features="sqrt",
            min_samples_leaf=1, class_weight=None, n_jobs=-1,
            random_state=random_state
        ),
        "HistGB": HistGradientBoostingClassifier(
            learning_rate=0.05, max_iter=400, max_depth=None,
            l2_regularization=0.0, random_state=random_state
        ),
        "GradBoost": GradientBoostingClassifier(
            n_estimators=400, learning_rate=0.05, max_depth=3, random_state=random_state
        ),
        "ExtraTrees": ExtraTreesClassifier(
            n_estimators=600, max_depth=None, max_features="sqrt",
            min_samples_leaf=1, n_jobs=-1, random_state=random_state
        ),
        "SVC": SVC(
            kernel="rbf", C=1.0, gamma="scale", probability=True,
            random_state=random_state
        ),
        "KNN": KNeighborsClassifier(
            n_neighbors=25, weights="distance", p=2
        ),
        "GaussianNB": GaussianNB()
    }
    # Optional: XGBoost (if installed)
    try:
        from xgboost import XGBClassifier
        models["XGBoost"] = XGBClassifier(
            n_estimators=700, learning_rate=0.05, max_depth=4,
            subsample=0.8, colsample_bytree=0.8, min_child_weight=1,
            reg_lambda=1.0, tree_method="hist",
            eval_metric="logloss", n_jobs=-1, random_state=random_state
        )
    except Exception:
        pass
    return models

# ============================================================
# === MAIN: Train, get probs, learn weights, threshold, evaluate
# ============================================================
# 0) Config
THRESH_STRATEGY = "cost"    # "cost" | "f1" | "balanced_accuracy" | "accuracy"
FP_COST, FN_COST = 1.0, 4.0
RANDOM_STATE = 42

# 1) Build models
models = get_model_zoo(RANDOM_STATE)

# 2) Fit on TRAIN, collect validation & test probabilities
val_probs = {}
test_probs = {}

for name, mdl in models.items():
    mdl.fit(X_train_res_scaled, y_train_res)
    val_probs[name]  = mdl.predict_proba(X_val_scaled)[:, 1]
    test_probs[name] = mdl.predict_proba(X_test_scaled)[:, 1]

# 3) Stack probs into matrices with fixed column order
model_order = list(val_probs.keys())   # keep insertion order
P_val  = np.column_stack([val_probs[m]  for m in model_order])
P_test = np.column_stack([test_probs[m] for m in model_order])

# 4) Learn convex (simplex) weights on VALIDATION (Brier-optimal)
w_star = solve_simplex_brier(P_val, y_val, iters=1500, lr=0.5)
weights = pd.Series(w_star, index=model_order).sort_values(ascending=False)
print("\nEnsemble simplex weights (sum=1):")
print(weights.round(4))
print("Sum:", round(weights.sum(), 6))

# Ensemble probabilities
val_ens  = P_val.dot(w_star)
test_ens = P_test.dot(w_star)

# 5) Choose threshold on VALIDATION ENSEMBLE
thr, sweep = find_best_threshold(y_val, val_ens,
                                 strategy=THRESH_STRATEGY,
                                 fp_cost=FP_COST, fn_cost=FN_COST)
print("\nChosen threshold (validation):", round(thr, 3))

# 6) Evaluate VAL + TEST for each model and for the ENSEMBLE (same threshold)
rows_val, rows_test = [], []

# Individual models
for name in model_order:
    rows_val.append({"Model": name, **evaluate_at_threshold(y_val,  val_probs[name],  thr)})
    rows_test.append({"Model": name, **evaluate_at_threshold(y_test, test_probs[name], thr)})

# Ensemble row
rows_val.append({"Model": "Ensemble", **evaluate_at_threshold(y_val,  val_ens,  thr)})
rows_test.append({"Model": "Ensemble", **evaluate_at_threshold(y_test, test_ens, thr)})

summary_val  = pd.DataFrame(rows_val)
summary_test = pd.DataFrame(rows_test)

# 7) Add cost columns and arrange
summary_val["ExpectedCost"] = summary_val["FP"]*FP_COST + summary_val["FN"]*FN_COST
summary_test["ExpectedCost"] = summary_test["FP"]*FP_COST + summary_test["FN"]*FN_COST
summary_val["CostPerCase"]  = summary_val["ExpectedCost"] / len(y_val)
summary_test["CostPerCase"] = summary_test["ExpectedCost"] / len(y_test)

cols_show = [
    "Model","Threshold","ROC_AUC","PR_AUC","Accuracy","Precision","Recall","F1",
    "Balanced_Accuracy","MCC","Brier","LogLoss","TN","FP","FN","TP","ExpectedCost","CostPerCase"
]
summary_val  = summary_val[cols_show].round(4).sort_values(["ExpectedCost","CostPerCase","PR_AUC"], ascending=[True,True,False])
summary_test = summary_test[cols_show].round(4).sort_values(["ExpectedCost","CostPerCase","PR_AUC"], ascending=[True,True,False])

# 8) Pretty display
print_or_display(summary_val, summary_test)

# 9) (Optional) CSV exports
# summary_val.to_csv("validation_summary_all_models.csv", index=False)
# summary_test.to_csv("test_summary_all_models.csv", index=False)
# pd.DataFrame({"model":model_order, "weight":w_star}).to_csv("ensemble_weights.csv", index=False)
# pd.DataFrame(sweep).to_csv("threshold_sweep_validation.csv", index=False)


/home/ppanta/puru_proj/proj_v0/hints6_v0/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(



Ensemble simplex weights (sum=1):
LogisticReg     0.6710
HistGB          0.2191
ExtraTrees      0.1099
RandomForest    0.0000
GradBoost       0.0000
SVC             0.0000
KNN             0.0000
GaussianNB      0.0000
XGBoost         0.0000
dtype: float64
Sum: 1.0

Chosen threshold (validation): 0.34


,Model,Threshold,ROC_AUC,PR_AUC,Accuracy,Precision,Recall,F1,Balanced_Accuracy,MCC,Brier,LogLoss,TN,FP,FN,TP,ExpectedCost,CostPerCase
9,Ensemble,0.34,0.7665,0.2457,0.8777,0.3158,0.2637,0.2874,0.6024,0.2222,0.0807,0.2789,830,52,67,24,320.0,0.3289
0,LogisticReg,0.34,0.7694,0.2606,0.8654,0.2778,0.2747,0.2762,0.6005,0.2020,0.0821,0.2884,817,65,66,25,329.0,0.3381
3,GradBoost,0.34,0.7173,0.2082,0.8510,0.2353,0.2637,0.2487,0.5877,0.1666,0.0897,0.3090,804,78,67,24,346.0,0.3556
4,ExtraTrees,0.34,0.7134,0.1959,0.8602,0.2353,0.2198,0.2273,0.5730,0.1506,0.0879,0.3054,817,65,71,20,349.0,0.3587
1,RandomForest,0.34,0.7120,0.1994,0.8469,0.2212,0.2527,0.2359,0.5805,0.1516,0.0898,0.3114,801,81,68,23,353.0,0.3628
5,SVC,0.34,0.6669,0.1763,0.8684,0.2239,0.1648,0.1899,0.5529,0.1217,0.0921,0.3262,830,52,76,15,356.0,0.3659
8,XGBoost,0.34,0.7069,0.1816,0.8571,0.1923,0.1648,0.1775,0.5467,0.1002,0.0920,0.3291,819,63,76,15,367.0,0.3772
2,HistGB,0.34,0.7192,0.2012,0.8684,0.1864,0.1209,0.1467,0.5332,0.0811,0.0908,0.3552,834,48,80,11,368.0,0.3782
6,KNN,0.34,0.6850,0.1638,0.7050,0.1449,0.4396,0.2180,0.5860,0.1111,0.1211,0.5081,646,236,51,40,440.0,0.4522
7,GaussianNB,0.34,0.6707,0.2012,0.5190,0.1297,0.7253,0.2200,0.6115,0.1300,0.3397,1.6445,439,443,25,66,543.0,0.5581


,Model,Threshold,ROC_AUC,PR_AUC,Accuracy,Precision,Recall,F1,Balanced_Accuracy,MCC,Brier,LogLoss,TN,FP,FN,TP,ExpectedCost,CostPerCase
0,LogisticReg,0.34,0.8000,0.3149,0.8664,0.3333,0.4444,0.3810,0.6769,0.3118,0.0777,0.2701,803,80,50,40,280.0,0.2878
9,Ensemble,0.34,0.8072,0.3229,0.8828,0.3667,0.3667,0.3667,0.6511,0.3021,0.0745,0.2593,826,57,57,33,285.0,0.2929
1,RandomForest,0.34,0.7781,0.2726,0.8664,0.3077,0.3556,0.3299,0.6370,0.2570,0.0814,0.2858,811,72,58,32,304.0,0.3124
4,ExtraTrees,0.34,0.7640,0.2485,0.8726,0.3152,0.3222,0.3187,0.6254,0.2484,0.0813,0.2822,820,63,61,29,307.0,0.3155
5,SVC,0.34,0.7059,0.2241,0.8849,0.3382,0.2556,0.2911,0.6023,0.2325,0.0858,0.3030,838,45,67,23,313.0,0.3217
2,HistGB,0.34,0.7471,0.2737,0.8828,0.3065,0.2111,0.2500,0.5812,0.1926,0.0825,0.3253,840,43,71,19,327.0,0.3361
3,GradBoost,0.34,0.7647,0.2549,0.8613,0.2632,0.2778,0.2703,0.5993,0.1938,0.0821,0.2845,813,70,65,25,330.0,0.3392
8,XGBoost,0.34,0.7586,0.2823,0.8705,0.2692,0.2333,0.2500,0.5844,0.1801,0.0810,0.2864,826,57,69,21,333.0,0.3422
6,KNN,0.34,0.7502,0.2183,0.7544,0.2078,0.5889,0.3072,0.6801,0.2373,0.1079,0.4411,681,202,37,53,350.0,0.3597
7,GaussianNB,0.34,0.6978,0.2126,0.5550,0.1389,0.7333,0.2336,0.6351,0.1566,0.3104,1.4876,474,409,24,66,505.0,0.5190


In [ ]:
import numpy as np
import pandas as pd

# ======================
# Try to import SHAP (optional)
# ======================
USE_SHAP = True
try:
    import shap  # pip install shap  OR  conda install -c conda-forge shap
except Exception:
    USE_SHAP = False

from sklearn.inspection import permutation_importance

# ======================
# 0) DataFrames for names
# ======================
X_train_df = pd.DataFrame(X_train_res_scaled, columns=features)
X_val_df   = pd.DataFrame(X_val_scaled,       columns=features)
X_test_df  = pd.DataFrame(X_test_scaled,      columns=features)

# Choose which split to explain (commonly validation)
X_explain = X_val_df
y_explain = y_val

# Background sample for SHAP (small subset of train for speed)
bg_n = min(256, len(X_train_df))
background = X_train_df.sample(bg_n, random_state=42)

# ======================
# 1) Model zoo & training (skip if you already have `models`)
# ======================
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, HistGradientBoostingClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

def get_model_zoo(random_state=42):
    zoo = {
        "LogisticReg": LogisticRegression(
            max_iter=1000, solver="liblinear", class_weight="balanced", random_state=random_state
        ),
        "RandomForest": RandomForestClassifier(
            n_estimators=600, max_depth=None, max_features="sqrt",
            min_samples_leaf=1, class_weight=None, n_jobs=-1, random_state=random_state
        ),
        "ExtraTrees": ExtraTreesClassifier(
            n_estimators=600, max_depth=None, max_features="sqrt",
            min_samples_leaf=1, n_jobs=-1, random_state=random_state
        ),
        "GradBoost": GradientBoostingClassifier(
            n_estimators=400, learning_rate=0.05, max_depth=3, random_state=random_state
        ),
        "HistGB": HistGradientBoostingClassifier(
            learning_rate=0.05, max_iter=400, random_state=random_state
        ),
        "SVC": SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=random_state),
        "KNN": KNeighborsClassifier(n_neighbors=25, weights="distance"),
        "GaussianNB": GaussianNB(),
    }
    try:
        from xgboost import XGBClassifier
        zoo["XGBoost"] = XGBClassifier(
            n_estimators=700, learning_rate=0.05, max_depth=4,
            subsample=0.8, colsample_bytree=0.8, min_child_weight=1,
            reg_lambda=1.0, tree_method="hist", eval_metric="logloss",
            n_jobs=-1, random_state=random_state
        )
    except Exception:
        pass
    return zoo

# Train models (comment this block if you already have `models`)
models = get_model_zoo(42)
for name, mdl in models.items():
    mdl.fit(X_train_res_scaled, y_train_res)

# ======================
# 2) Explainers / importance (SHAP with new API; permutation fallback)
# ======================
def make_explainer(model, model_name, background_df):
    """
    Robust SHAP explainer factory compatible with newer/older SHAP.
    Uses maskers for new API; falls back gracefully as needed.
    """
    import shap
    name = str(model_name).lower()

    # Build masker (new SHAP API)
    try:
        masker = shap.maskers.Independent(background_df)
    except Exception:
        masker = background_df

    # Tree-based → TreeExplainer
    if any(tok in name for tok in ["randomforest", "extratrees", "gradboost", "histgb", "xgboost", "xgb"]):
        return shap.TreeExplainer(model)

    # Linear models → LinearExplainer
    if "logistic" in name or "linear" in name:
        try:
            return shap.LinearExplainer(model, masker)
        except TypeError:
            return shap.LinearExplainer(model, background_df)

    # Generic fallback
    try:
        return shap.Explainer(model, masker)
    except TypeError:
        # very old fallback
        return shap.KernelExplainer(model.predict_proba, background_df)

def compute_shap_importance(model, model_name, X_bg, X_data, top_k=None):
    """
    Returns (importance_df, shap_values_matrix_for_class1).
    importance_df has columns: Feature, <model>_MeanAbsSHAP, <model>_Rank
    """
    explainer = make_explainer(model, model_name, X_bg)
    shap_out = explainer(X_data)  # shap.Explanation (new) or array/list (old)

    # Normalize to (n_samples, n_features) for positive class
    if hasattr(shap_out, "values"):  # Explanation
        values = shap_out.values
    else:
        values = shap_out

    # Handle multi-class outputs (n, n_features, n_classes)
    if isinstance(values, np.ndarray) and values.ndim == 3 and values.shape[-1] >= 2:
        values = values[..., 1]
    if isinstance(values, list) and len(values) >= 2 and isinstance(values[1], np.ndarray):
        values = values[1]

    mean_abs = np.abs(values).mean(axis=0)
    col_mean = f"{model_name}_MeanAbsSHAP"
    col_rank = f"{model_name}_Rank"
    imp_df = (pd.DataFrame({"Feature": X_data.columns, col_mean: mean_abs})
                .sort_values(col_mean, ascending=False))
    if top_k:
        imp_df = imp_df.head(top_k).copy()
    imp_df[col_rank] = np.arange(1, len(imp_df) + 1)
    return imp_df, values

def compute_perm_importance(model, model_name, X_ref, y_ref, n_repeats=10, random_state=42, top_k=None):
    """
    Model-agnostic permutation importance (mean abs decrease in proba MSE proxy).
    Returns (importance_df, None). Uses scoring on predicted proba for class 1.
    """
    # Use validation split for permutation importance
    r = permutation_importance(
        model, X_ref, y_ref,
        scoring="neg_brier_score",  # aligns with probability quality
        n_repeats=n_repeats, random_state=random_state, n_jobs=-1
    )
    # r.importances_mean can be negative due to noise; use abs to rank magnitude
    scores = np.abs(r.importances_mean)
    col_mean = f"{model_name}_PermImportance"
    col_rank = f"{model_name}_Rank"
    imp_df = (pd.DataFrame({"Feature": X_ref.columns, col_mean: scores})
                .sort_values(col_mean, ascending=False))
    if top_k:
        imp_df = imp_df.head(top_k).copy()
    imp_df[col_rank] = np.arange(1, len(imp_df) + 1)
    return imp_df, None

# ======================
# 3) Run for all models
# ======================
model_names = list(models.keys())
per_model_tables = {}
raw_values = {}  # SHAP arrays if SHAP used

for name in model_names:
    if USE_SHAP:
        imp_df, shap_vals = compute_shap_importance(models[name], name, background, X_explain, top_k=None)
        per_model_tables[name] = imp_df.reset_index(drop=True)
        raw_values[name] = shap_vals  # (n_samples, n_features)
    else:
        imp_df, _ = compute_perm_importance(models[name], name, X_explain, y_explain, n_repeats=10)
        per_model_tables[name] = imp_df.reset_index(drop=True)
        raw_values[name] = None

# ======================
# 4) Build consensus tables
# ======================
consensus = pd.DataFrame({"Feature": features})

# Merge each model’s (MeanAbsSHAP or PermImportance) and Rank
for name, imp_df in per_model_tables.items():
    val_col = f"{name}_MeanAbsSHAP" if USE_SHAP else f"{name}_PermImportance"
    consensus = consensus.merge(imp_df[["Feature", val_col, f"{name}_Rank"]], on="Feature", how="left")

# Fill missing (if top_k used; here we used all)
fill_vals = {}
for name in model_names:
    fill_vals[f"{name}_Rank"] = np.nan
    fill_vals[f"{name}_MeanAbsSHAP" if USE_SHAP else f"{name}_PermImportance"] = 0.0
consensus = consensus.fillna(fill_vals)

# Average Rank (smaller = better). Penalize missing ranks by using len+1.
adj_ranks = []
for name in model_names:
    r = consensus[f"{name}_Rank"].copy()
    r = r.fillna(len(consensus) + 1)
    adj_ranks.append(r)
consensus["Avg_Rank"] = np.mean(np.vstack(adj_ranks), axis=0)

# Normalized importance per model to [0,1], then average
norm_cols = []
for name in model_names:
    base_col = f"{name}_MeanAbsSHAP" if USE_SHAP else f"{name}_PermImportance"
    maxv = consensus[base_col].max()
    ncol = f"{name}_NormImp"
    consensus[ncol] = consensus[base_col] / maxv if maxv > 0 else 0.0
    norm_cols.append(ncol)

consensus["Avg_NormImportance"] = consensus[norm_cols].mean(axis=1)

# Two sorted views
consensus_by_rank = consensus.sort_values("Avg_Rank").reset_index(drop=True)
consensus_by_norm = consensus.sort_values("Avg_NormImportance", ascending=False).reset_index(drop=True)

print("\nTop 30 by Average Rank across models:")
print(consensus_by_rank[["Feature","Avg_Rank"] + [f"{name}_Rank" for name in model_names]].head(30))

print("\nTop 30 by Average Normalized Importance across models:")
print(consensus_by_norm[["Feature","Avg_NormImportance"] + norm_cols].head(30))

# ======================
# 5) (Optional) Save CSVs
# ======================
# for name, df_imp in per_model_tables.items():
#     df_imp.to_csv(f"importance_{name}.csv", index=False)
# consensus_by_rank.to_csv("consensus_by_rank.csv", index=False)
# consensus_by_norm.to_csv("consensus_by_avg_norm.csv", index=False)

# ======================
# 6) (Optional) Quick SHAP plots if SHAP is available
# ======================
# if USE_SHAP:
#     import matplotlib.pyplot as plt
#     # Example: beeswarm plot for a tree model (RandomForest) and for LR
#     if raw_values.get("RandomForest") is not None:
#         shap.summary_plot(raw_values["RandomForest"], X_explain, show=True)
#     if raw_values.get("LogisticReg") is not None:
#         shap.summary_plot(raw_values["LogisticReg"], X_explain, show=True)


/home/ppanta/puru_proj/proj_v0/hints6_v0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
